In [ ]:
# This code calls the Spotify API and gets the playlists of the owner of the access token
import requests
import json

# Replace this values with access token
authkey = "REPLACE_WITH_YOUR_ACCESS_TOKEN"

url = f'https://api.spotify.com/v1/me/playlists'
headers = {
    'Authorization': f'Bearer {authkey}',
}
params = {
    'limit': 50,
    'offset': 0,
}
response = requests.get(url, headers=headers, params=params)
if response.status_code == 200:
    playlists_data = response.json()

    with open('playlists.json', 'w', encoding='utf-8') as json_file:
        json.dump(playlists_data, json_file, ensure_ascii=False, indent=4)

    print(f"Result saved to 'playlists.json'")
else:
    print(f"Error: {response.status_code} - {response.text}")
    print(response.headers)

In [ ]:
# This code calls the Spotify api and gets the items from a playlist and saves them to a json file
import requests
import json

# Find the playlist ID from the 'playlists.json' file from the prevous step, 
# and replace 'REPLACE_WITH_YOUR_PLAYLIST_ID' with the actual playlist ID, 
# this is "2byXPx9exNcW26Xpo3yXcV" from "id": "2byXPx9exNcW26Xpo3yXcV",

playlist_id = 'REPLACE_WITH_YOUR_PLAYLIST_ID'
access_token = "REPLACE_WITH_YOUR_ACCESS_TOKEN"

def get_playlist_items(playlist_id, access_token, offset=0, limit=50, file_count=1):
    url = f'https://api.spotify.com/v1/playlists/{playlist_id}/tracks'
    headers = {
        'Authorization': f'Bearer {access_token}',
    }
    params = {
        'limit': limit,
        'offset': offset,
    }
    response = requests.get(url, headers=headers, params=params)
    if response.status_code == 200:
        items_data = response.json()
        filename = f'PLAYLIST_items_{file_count}.json'
        with open(filename, 'w', encoding='utf-8') as json_file:
            json.dump(items_data, json_file, ensure_ascii=False, indent=4)
        if len(items_data.get('items', [])) == limit:
            get_playlist_items(playlist_id, access_token, offset + limit, limit, file_count + 1)
    else:
        print(f"Error: {response.status_code} - {response.text}")
        print(response.headers)

get_playlist_items(playlist_id, access_token)

In [ ]:
# This code gets the unique artist IDs from all the playlist items JSON files, and saves them to a text file
import json
import os
artist_ids = []
unique_artist_ids = set()

# Replace this with the directory where the PLAYLIST_items_ files are located
directory = 'C:\\Users\\Music\\Desktop\\PROJECTS\\Spotify Project\\PythonNotebooksForPrepopulation\\'

for filename in os.listdir(directory):
    if filename.startswith('PLAYLIST_items_') and filename.endswith('.json'):
        file_path = os.path.join(directory, filename)
        with open(file_path, 'r') as file:
            data = json.load(file)
        for item in data.get('items', []):
            if 'track' in item and 'artists' in item['track']:
                for artist in item['track']['artists']:
                    if 'id' in artist and artist['id'] not in unique_artist_ids:
                        artist_ids.append(artist['id'])
                        unique_artist_ids.add(artist['id'])

output_file_path = 'artist_ids_list.txt'
with open(output_file_path, 'w') as output_file:
    json.dump(artist_ids, output_file)

print(artist_ids)

In [ ]:
# OPTIONAL - if you have two lists of artists in two different playlists, you can merge them and remove duplicates using the following code:
import json
def load_list_from_file(file_path):
    try:
        with open(file_path, 'r') as file:
            return json.loads(file.read())
    except FileNotFoundError:
        return []

# Paths to the two text files containing lists
file_path1 = 'artist_ids_list1.txt'
file_path2 = 'artist_ids_list2.txt'

list1 = load_list_from_file(file_path1)
list2 = load_list_from_file(file_path2)
merged_list = list(set(list1 + list2))
output_file_path = 'merged_artist_ids_list.txt'
with open(output_file_path, 'w') as output_file:
    json.dump(merged_list, output_file)
print(merged_list)